In [ ]:
import cv2
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import torch
from pythonopenpose import pyopenpose as op
import matplotlib.patches as mpatches
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import imageio

Define paths

In [ ]:
c233_path = r'C:\Users\aksh0\Desktop\Hackenza\Quidich-HACKATHON-25\233_im'
c235_path = r'C:\Users\aksh0\Desktop\Hackenza\Quidich-HACKATHON-25\235_im'
intrinsic_path = r'C:\Users\aksh0\Desktop\Hackenza\Quidich-HACKATHON-25\intrinsic.json'
extrinsic_path = r'C:\Users\aksh0\Desktop\Hackenza\Quidich-HACKATHON-25\extrinsic.json'
output_dir = r'C:\Users\aksh0\Desktop\Hackenza\Quidich-HACKATHON-25\processed_frames'
os.makedirs(output_dir, exist_ok=True)

Load intrinsic and extrinsic data

In [ ]:
with open(intrinsic_path, 'r') as f:
    intrinsic_data = json.load(f)

In [ ]:
with open(extrinsic_path, 'r') as f:
    extrinsic_data = json.load(f)

Extract camera matrices and distortion coefficients

In [ ]:
K1 = np.array(intrinsic_data['C233']['camera_matrix'])
D1 = np.array(intrinsic_data['C233']['distortion_coefficients'])
K2 = np.array(intrinsic_data['C235']['camera_matrix'])
D2 = np.array(intrinsic_data['C235']['distortion_coefficients'])

Extract rotation and translation matrices

In [ ]:
R1 = cv2.Rodrigues(np.array(extrinsic_data['rotation_vectors']['C233']))[0]
T1 = np.array(extrinsic_data['translation_vectors']['C233']).reshape(3, 1)
R2 = cv2.Rodrigues(np.array(extrinsic_data['rotation_vectors']['C235']))[0]
T2 = np.array(extrinsic_data['translation_vectors']['C235']).reshape(3, 1)

Compute projection matrices

In [ ]:
P1 = K1 @ np.hstack((R1, T1))
P2 = K2 @ np.hstack((R2, T2))

Initialize OpenPose

In [ ]:
params = {
    "model_folder": "models/",
    "model_pose": "BODY_25",
    "net_resolution": "656x368",
    "number_people_max": 1,
    "keypoint_scale": 3
}

Starting OpenPose

In [ ]:
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

Define body parts by their index in BODY_25 model

In [ ]:
BODY_PARTS = {
    "Head": [0, 1, 2, 3, 4, 15, 16, 17, 18],  # Nose, Eyes, Ears
    "Torso": [1, 8, 5, 2, 12, 9, 1],  # Neck, Mid-hip, Shoulders, Hips
    "LeftArm": [5, 6, 7],  # Left Shoulder, Elbow, Wrist
    "RightArm": [2, 3, 4],  # Right Shoulder, Elbow, Wrist
    "LeftLeg": [12, 13, 14],  # Left Hip, Knee, Ankle
    "RightLeg": [9, 10, 11]  # Right Hip, Knee, Ankle
}

Define color for each body part

In [ ]:
COLORS = {
    "Head": (255, 255, 0),       # Yellow
    "Torso": (0, 0, 255),        # Blue
    "LeftArm": (0, 255, 0),      # Green
    "RightArm": (0, 255, 0),     # Green
    "LeftLeg": (255, 0, 0),      # Red
    "RightLeg": (255, 0, 0)      # Red
}

3D visualization connections

In [ ]:
SKELETON_CONNECTIONS_3D = {
    "Head": [(0, 1), (1, 2), (1, 5), (2, 3), (3, 4), (5, 6), (6, 7), (1, 8), (1, 8), (15, 17), (16, 18)],
    "Torso": [(1, 8), (8, 9), (8, 12), (1, 2), (1, 5), (2, 9), (5, 12)],
    "LeftArm": [(5, 6), (6, 7)],
    "RightArm": [(2, 3), (3, 4)],
    "LeftLeg": [(12, 13), (13, 14)],
    "RightLeg": [(9, 10), (10, 11)]
}

Function to triangulate a point between two camera views

In [ ]:
def triangulate_point(P1, P2, point1, point2):
    A = np.array([
        point1[0] * P1[2] - P1[0],
        point1[1] * P1[2] - P1[1],
        point2[0] * P2[2] - P2[0],
        point2[1] * P2[2] - P2[1]
    ])
    _, _, V = np.linalg.svd(A)
    X = V[-1]
    return X[:3] / X[3]

Function to draw skeletal structure on 2D images

In [ ]:
def draw_skeleton_2d(image, keypoints, threshold=0.5):
    # Draw lines for each body part
    for part_name, indices in BODY_PARTS.items():
        color = COLORS[part_name]
        # Connect consecutive points in each body part
        for i in range(len(indices) - 1):
            idx1, idx2 = indices[i], indices[i + 1]
            if keypoints[idx1, 2] > threshold and keypoints[idx2, 2] > threshold:
                pt1 = (int(keypoints[idx1, 0]), int(keypoints[idx1, 1]))
                pt2 = (int(keypoints[idx2, 0]), int(keypoints[idx2, 1]))
                cv2.line(image, pt1, pt2, color, 2)
        
        # Draw keypoints
        for idx in indices:
            if keypoints[idx, 2] > threshold:
                cv2.circle(image, (int(keypoints[idx, 0]), int(keypoints[idx, 1])), 
                           3, color, -1)
    
    return image

Process a single frame

In [ ]:
def process_frame(frame_number):
    # Construct file names
    file1 = f"HPUP_033_1_1_1_L_CAM-05_{frame_number:07d}.jpeg"
    file2 = f"HPUP_033_1_1_1_L_CAM-02_{frame_number:07d}.jpeg"

    # Read frames from both cameras
    frame1 = cv2.imread(os.path.join(c233_path, file1))
    frame2 = cv2.imread(os.path.join(c235_path, file2))
    if frame1 is None or frame2 is None:
        print(f"Frame {frame_number} missing in one of the cameras.")
        return None

    # Process with OpenPose
    datum1 = op.Datum()
    datum2 = op.Datum()
    
    datum1.cvInputData = frame1
    datum2.cvInputData = frame2
    
    opWrapper.emplaceAndPop(op.VectorDatum([datum1]))
    opWrapper.emplaceAndPop(op.VectorDatum([datum2]))
    
    # Check if any poses were detected
    if datum1.poseKeypoints is None or datum2.poseKeypoints is None:
        print(f"No poses detected in frame {frame_number}")
        return None
    
    if len(datum1.poseKeypoints) == 0 or len(datum2.poseKeypoints) == 0:
        print(f"No poses detected in frame {frame_number}")
        return None
    
    # Get the first person's keypoints
    keypoints1 = datum1.poseKeypoints[0]
    keypoints2 = datum2.poseKeypoints[0]
    
    # Draw skeletons on 2D images
    frame1_with_skeleton = draw_skeleton_2d(frame1.copy(), keypoints1)
    frame2_with_skeleton = draw_skeleton_2d(frame2.copy(), keypoints2)
    
    # Save processed frames
    cv2.imwrite(os.path.join(output_dir, f"cam233_frame_{frame_number}.jpg"), frame1_with_skeleton)
    cv2.imwrite(os.path.join(output_dir, f"cam235_frame_{frame_number}.jpg"), frame2_with_skeleton)
    
    # Initialize 3D points list and body part labels
    points_3d = []
    landmark_types = []
    
    # Triangulate all keypoints
    for i in range(keypoints1.shape[0]):
        # Check confidence threshold
        if keypoints1[i, 2] > 0.5 and keypoints2[i, 2] > 0.5:
            point1 = keypoints1[i, :2]
            point2 = keypoints2[i, :2]
            
            # Triangulate 3D point
            point_3d = triangulate_point(P1, P2, point1, point2)
            points_3d.append(point_3d)
            
            # Determine which body part this keypoint belongs to
            part_type = "Other"
            for part_name, indices in BODY_PARTS.items():
                if i in indices:
                    part_type = part_name
                    break
            
            landmark_types.append(part_type)
        else:
            # Add a placeholder for low confidence points
            points_3d.append(np.array([0, 0, 0]))
            landmark_types.append("Low Confidence")
    
    return np.array(points_3d), landmark_types

Process all frames

In [ ]:
print("Starting frame processing...")
all_3d_points = []
all_landmark_types = []
frame_range = range(350, 507, 1)

In [ ]:
for frame_number in frame_range:
    if frame_number % 10 == 0:
        print(f"Processing frame {frame_number}")
    
    result = process_frame(frame_number)
    if result is not None:
        points_3d, landmark_types = result
        all_3d_points.append(points_3d)
        all_landmark_types.append(landmark_types)

In [ ]:
print(f"Processed {len(all_3d_points)} frames successfully")

Function to draw skeleton in 3D plot with improved rendering

In [ ]:
def draw_skeleton_3d(ax, points, landmark_types):
    # Plot connections between keypoints for each body part
    for part_name, connections in SKELETON_CONNECTIONS_3D.items():
        color_map = {"Head": 'yellow', "Torso": 'blue', "LeftArm": 'green', 
                    "RightArm": 'green', "LeftLeg": 'red', "RightLeg": 'red'}
        
        for connection in connections:
            idx1, idx2 = connection
            if idx1 < len(points) and idx2 < len(points):
                # Check if points are valid (not placeholder zeros)
                if not np.all(points[idx1] == 0) and not np.all(points[idx2] == 0):
                    ax.plot([points[idx1][0], points[idx2][0]],
                           [points[idx1][1], points[idx2][1]],
                           [points[idx1][2], points[idx2][2]],
                           color=color_map[part_name], linewidth=3, alpha=0.7)
    
    # Plot keypoints with colors based on body part
    for i, (point, part_type) in enumerate(zip(points, landmark_types)):
        if part_type != "Low Confidence" and not np.all(point == 0):
            color_map = {"Head": 'yellow', "Torso": 'blue', "LeftArm": 'green', 
                        "RightArm": 'green', "LeftLeg": 'red', "RightLeg": 'red', "Other": 'gray'}
            
            ax.scatter(point[0], point[1], point[2], 
                      color=color_map.get(part_type, 'gray'), 
                      s=30, edgecolors='black', alpha=0.8)

Create a higher quality 3D animation with improved renderer

In [ ]:
def create_animation():
    print("Creating 3D animation...")
    fig = plt.figure(figsize=(12, 12), dpi=150)
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111, projection='3d')
    
    # Set up the 3D plot style
    ax.set_facecolor('whitesmoke')
    fig.patch.set_facecolor('white')
    
    # Adjust the viewing angle for better perspective
    ax.view_init(elev=20, azim=60)
    
    # Create empty frames list for animation
    frames = []
    
    # Calculate bounds for consistent scaling
    all_x = []
    all_y = []
    all_z = []
    for points in all_3d_points:
        valid_points = points[~np.all(points == 0, axis=1)]
        if len(valid_points) > 0:
            all_x.extend(valid_points[:, 0])
            all_y.extend(valid_points[:, 1])
            all_z.extend(valid_points[:, 2])
    
    x_range = [min(all_x), max(all_x)]
    y_range = [min(all_y), max(all_y)]
    z_range = [min(all_z), max(all_z)]
    
    # Buffer for better viewing
    buffer = 0.1
    x_range = [x_range[0] - buffer * (x_range[1] - x_range[0]), 
               x_range[1] + buffer * (x_range[1] - x_range[0])]
    y_range = [y_range[0] - buffer * (y_range[1] - y_range[0]), 
               y_range[1] + buffer * (y_range[1] - y_range[0])]
    z_range = [z_range[0] - buffer * (z_range[1] - z_range[0]), 
               z_range[1] + buffer * (z_range[1] - z_range[0])]
    
    for frame_idx in range(len(all_3d_points)):
        if frame_idx % 10 == 0:
            print(f"Rendering frame {frame_idx + 350}")
        
        ax.clear()
        
        # Set fixed axis limits for consistency
        ax.set_xlim(x_range)
        ax.set_ylim(y_range)
        ax.set_zlim(z_range)
        
        # Setup axes
        ax.set_xlabel('X', fontsize=14)
        ax.set_ylabel('Y', fontsize=14)
        ax.set_zlabel('Z', fontsize=14)
        ax.set_title(f'3D Pose Estimation - Frame {frame_idx + 350}', fontsize=16)
        
        # Draw the skeleton
        draw_skeleton_3d(ax, all_3d_points[frame_idx], all_landmark_types[frame_idx])
        
        # Add legend
        head_patch = mpatches.Patch(color='yellow', label='Head')
        torso_patch = mpatches.Patch(color='blue', label='Torso')
        arm_patch = mpatches.Patch(color='green', label='Arms')
        leg_patch = mpatches.Patch(color='red', label='Legs')
        ax.legend(handles=[head_patch, torso_patch, arm_patch, leg_patch], 
                 loc='upper right', fontsize=12)
        
        # Render frame
        fig.tight_layout()
        canvas.draw()
        
        # Convert to image
        img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        
        frames.append(img)
    
    # Save the animation as high quality video
    print("Saving animation...")
    imageio.mimsave(os.path.join(output_dir, '3d_pose_animation_improved.mp4'), 
                   frames, fps=10, quality=8)
    
    # Also save as gif for wider compatibility
    imageio.mimsave(os.path.join(output_dir, '3d_pose_animation_improved.gif'), 
                   frames, fps=5)
    
    print(f"Animation saved to {output_dir}")

Create still image visualization for single frame

In [ ]:
def visualize_single_frame():
    if len(all_3d_points) == 0:
        print("No frames processed successfully.")
        return
    
    # Create a high-quality figure for a single frame
    fig = plt.figure(figsize=(12, 12), dpi=150)
    ax = fig.add_subplot(111, projection='3d')
    
    # Set up the 3D plot style
    ax.set_facecolor('whitesmoke')
    fig.patch.set_facecolor('white')
    
    # Adjust the viewing angle for better perspective
    ax.view_init(elev=20, azim=60)
    
    # Draw the first frame
    draw_skeleton_3d(ax, all_3d_points[0], all_landmark_types[0])
    
    # Set axis labels and title
    ax.set_xlabel('X', fontsize=14)
    ax.set_ylabel('Y', fontsize=14)
    ax.set_zlabel('Z', fontsize=14)
    ax.set_title('3D Pose Estimation with Enhanced Rendering', fontsize=16)
    
    # Add legend
    head_patch = mpatches.Patch(color='yellow', label='Head')
    torso_patch = mpatches.Patch(color='blue', label='Torso')
    arm_patch = mpatches.Patch(color='green', label='Arms')
    leg_patch = mpatches.Patch(color='red', label='Legs')
    ax.legend(handles=[head_patch, torso_patch, arm_patch, leg_patch], 
             loc='upper right', fontsize=12)
    
    # Save high-resolution image
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, '3d_pose_single_frame.png'), dpi=300)
    plt.show()

Run the visualization and animation functions

In [ ]:
if len(all_3d_points) > 0:
    print("Generating visualizations...")
    visualize_single_frame()
    create_animation()
    print("Processing complete!")
else:
    print("No frames were successfully processed. Check your paths and OpenPose installation.")

Clean up

In [ ]:
cv2.destroyAllWindows()